## MinIO -> (pdfs)

minio = "^7.2.7"
sentence-transformers = "^3.0.1"
faiss-cpu = "^1.8.0.post1"

Install minio image and run its created container  

In [1]:
!docker pull -q quay.io/minio/minio

quay.io/minio/minio:latest


In [2]:
!docker run -d -p 9000:9000 -p 9001:9001 --name minio -e "MINIO_ROOT_USER=minioadmin" -e "MINIO_ROOT_PASSWORD=password" -v D:\data:/data quay.io/minio/minio server /data --console-address ":9001"

f3e67b18f774a8a2e5f6cec203170ee28c2053dc3b8578ab3572e124a6aaadd4


Connenct to the minio

In [4]:
from minio import Minio, S3Error

# create client with access key and secret key with specific region.
client = Minio(
    endpoint="127.0.0.1:9000",  # 9090
    access_key="minioadmin",
    secret_key="password",
    secure=False,
)

In [8]:
"""
might be a solution to search api problem
Create client with custom HTTP client using proxy server.
# import urllib3
# client = Minio(
#     "SERVER:PORT",
#     access_key="ACCESS_KEY",
#     secret_key="SECRET_KEY",
#     secure=True,
#     http_client=urllib3.ProxyManager(
#         "https://PROXYSERVER:PROXYPORT/",
#         timeout=urllib3.Timeout.DEFAULT_TIMEOUT,
#         cert_reqs="CERT_REQUIRED",
#         retries=urllib3.Retry(
#             total=5,
#             backoff_factor=0.2,
#             status_forcelist=[500, 502, 503, 504],
#         ),
#     ),
# )
"""

'\nmight be a solution to search api problem\nCreate client with custom HTTP client using proxy server.\n# import urllib3\n# client = Minio(\n#     "SERVER:PORT",\n#     access_key="ACCESS_KEY",\n#     secret_key="SECRET_KEY",\n#     secure=True,\n#     http_client=urllib3.ProxyManager(\n#         "https://PROXYSERVER:PROXYPORT/",\n#         timeout=urllib3.Timeout.DEFAULT_TIMEOUT,\n#         cert_reqs="CERT_REQUIRED",\n#         retries=urllib3.Retry(\n#             total=5,\n#             backoff_factor=0.2,\n#             status_forcelist=[500, 502, 503, 504],\n#         ),\n#     ),\n# )\n'

In [5]:
BUCKET_NAME = "test"

if not client.bucket_exists(BUCKET_NAME):
    client.make_bucket(BUCKET_NAME)

client.list_buckets()

[Bucket('test')]

In [6]:
objects = client.list_objects(BUCKET_NAME)
for obj in objects:
    print(obj)

In [2]:
import shutil

shutil.copyfile("../../data/moodle/files/module-111237.pdf", "module-111237.pdf")

'module-111237.pdf'

In [7]:
import os

try:
    file_path = "module-111237.pdf"
    with open(file_path, "rb") as file:
        file_stat = os.stat(file_path)

        client.put_object(
            bucket_name=BUCKET_NAME,
            object_name=file_path,
            data=file,
            length=file_stat.st_size,
            content_type="application/pdf",
        )
except S3Error as e:
    print("error occurred.", e)

In [8]:
objects = client.list_objects(BUCKET_NAME)
for obj in objects:
    print(obj)

In [9]:
os.remove(file_path)

In [10]:
# Get data of an object.
try:
    client.fget_object(BUCKET_NAME, file_path, file_path)
except Exception:
    pass

In [11]:
objects = client.list_objects(BUCKET_NAME)
for obj in objects:
    print(obj)

## pymupdf4llm -> (md)

In [57]:
from tqdm.notebook import tqdm
from pymupdf4llm import process_document, join_chunks

entries = [f for f in os.listdir(".") if f.endswith(".pdf")]
processed_documents = [process_document(entry) for entry in entries]

In [58]:
texts = []
for result in tqdm(processed_documents, total=len(entries)):
    texts.append(join_chunks(result))

  0%|          | 0/2 [00:00<?, ?it/s]

In [59]:
texts[0]

"<h2><b>Deep Learning for Search</b></h2>\n<h2><b>(summer 2024)</b></h2>\n\nAlbert Nasybullin\n\n-----\n\n● Masters in Robotics and Computer\nVision, Innopolis (2021)\n\n● PhD student, Deep Learning and\nBrain-Computer Interfaces\n\n● Teaching: Probability & Statistics,\nDigital Signal Processing, Machine\nLearning, Deep Learning (Innopolis\nUniversity, SPSU)\n\n● Machine Learning for Banking\n\n● Nowadays, MTS. Machine Learning\nEngineer (NLP & Search)\n tlg: @levshaazz\nemail: levshaazz@gmail.com\n\n-----\n\n<b>The Course Aims</b>\n\n● Natural Language Processing\n\n● Similarity Search\n\n● Deep Learning\n\n● Machine Learning System Design\n\n● Metrics of Search\n\n● Risk approach to Machine Learning\n\n● Optimization of Neural Nets and Data\nstorage\n\n● Data quality and augmentation\n\n● Retrieval Augmented Generation\n\n-----\n\n<b>The Course Aims</b>\n\n● Natural Language Processing\n\n● Similarity Search\n\n● Deep Learning\n\n● Machine Learning System Design\n\n● Metrics of Sear

## Preprocess -> (str)

In [16]:
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer  # , TreebankWordTokenizer
import unicodedata

nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

In [60]:
def normalize(text: str) -> str:
    return unicodedata.normalize("NFKC", text)


def to_lowercase(text: str) -> str:
    return text.lower()


def remove_punctuation(text: str) -> str:
    return re.sub(r"[^\w\s]", "", text)


def remove_html(text: str) -> str:
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text(separator=" ")


def remove_markdown(text: str) -> str:
    # TODO: ...
    return text


def remove_urls(text: str) -> str:
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    return re.sub(url_pattern, "", text)


def remove_stop_words(words: list[str]) -> list[str]:
    stop_words = set(stopwords.words("english")) | set(stopwords.words("russian"))
    filtered_words = [word for word in words if word.casefold() not in stop_words]
    return filtered_words

In [61]:
def test_preprocessing():
    assert normalize("𝑃𝑜𝑠𝑖𝑡𝑖𝑣𝑒𝑠") == "Positives"
    assert to_lowercase("Omg Hiiiii") == "omg hiiiii"
    assert remove_html("<h2><b>Deep Learning for Search</b></h2>") == "Deep Learning for Search"
    assert remove_punctuation("Hello, quite! A lot? :_+") == "Hello quite A lot _"
    # assert remove_markdown("") == ""
    assert remove_urls("Here is the link https://hotel.innopolis.university/dokumenty/ !") == "Here is the link  !"
    # assert remove_stop_words([]) == []


test_preprocessing()

In [62]:
def preprocess(text: str, tokenizer) -> str:
    text = normalize(text)  # . normalize
    text = to_lowercase(text)  # to lower case (think of Names etc)
    text = remove_html(text)  # remove tags
    text = remove_punctuation(text)  # remove punctuation
    # text = remove_markdown(text)
    text = remove_urls(text)  # remove urls

    # TOKENIZE #
    words = tokenizer.tokenize(text)
    words = remove_stop_words(words)

    return " ".join(words)

In [63]:
texts = [preprocess(text, WordPunctTokenizer()) for text in texts]

In [64]:
texts[0]

'deep learning search summer 2024 albert nasybullin masters robotics computer vision innopolis 2021 phd student deep learning braincomputer interfaces teaching probability statistics digital signal processing machine learning deep learning innopolis university spsu machine learning banking nowadays mts machine learning engineer nlp search tlg levshaazz email levshaazzgmailcom course aims natural language processing similarity search deep learning machine learning system design metrics search risk approach machine learning optimization neural nets data storage data quality augmentation retrieval augmented generation course aims natural language processing similarity search deep learning machine learning system design metrics search risk approach machine learning optimization neural nets data storage data quality augmentation retrieval augmented generation end course able design develop deep learningbased search application course highlights tuesdays 1740 2050 314 start june 4 end july 1

## Junking -> (junks)

In [66]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [67]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=150,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)

In [71]:
chunks = text_splitter.split_text(texts[0])

In [72]:
chunks

['deep learning search summer 2024 albert nasybullin masters robotics computer vision innopolis 2021 phd student deep learning braincomputer interfaces',
 'interfaces teaching probability statistics digital signal processing machine learning deep learning innopolis university spsu machine learning',
 'machine learning banking nowadays mts machine learning engineer nlp search tlg levshaazz email levshaazzgmailcom course aims natural language',
 'natural language processing similarity search deep learning machine learning system design metrics search risk approach machine learning optimization',
 'optimization neural nets data storage data quality augmentation retrieval augmented generation course aims natural language processing similarity',
 'similarity search deep learning machine learning system design metrics search risk approach machine learning optimization neural nets data storage',
 'nets data storage data quality augmentation retrieval augmented generation end course able desig

## Embeddings -> (vectors)

In [73]:
from sentence_transformers import SentenceTransformer

# MODEL_NAME = 'all-mpnet-base-v2'  # SOTA
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

c:\Users\PodYapolsky\AppData\Local\pypoetry\Cache\virtualenvs\search-experiments-AGuved54-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [74]:
embeddings = model.encode(chunks)

c:\Users\PodYapolsky\AppData\Local\pypoetry\Cache\virtualenvs\search-experiments-AGuved54-py3.11\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [76]:
embeddings.shape

(34, 384)

## FAISS

Check for normalization of embeddings

In [95]:
import numpy as np

i = 0
for embedding in embeddings:
    if abs(np.linalg.norm(embeddings[1]) - 1) > 1e-5:
        i += 1

print(i)

0


Since vectors are normalized, IP (Inner Product) will work as cosine

https://github.com/facebookresearch/faiss/wiki/Faiss-indexes
https://github.com/facebookresearch/faiss/issues/95


In [99]:
import faiss

n, d = embeddings.shape
index = faiss.IndexFlatIP(d)

In [100]:
index.is_trained

True

In [112]:
index.add(embeddings)

Query

In [116]:
query_embedding = model.encode(["Phd student Albert Nasibullin"])

In [117]:
D, I = index.search(query_embedding, 3)  # noqa: E741

First chunk is the closest as we expected

In [119]:
print(D, I)

[[0.36785334 0.21152827 0.1874401 ]] [[0 7 2]]
